In [ ]:
import glob
import os
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time

In [ ]:
from torchvision.datasets.vision import VisionDataset
from torch.utils.data import IterableDataset
# from torchvision.datasets.video_utils import VideoClips
from video_clip import VideoClips
import torch.utils.data as data
from bat_seg_models import ThreeLayerSemSegNetWideView, UNET, UNETTraditional
from frame_augmentors import MaskNormalize, Mask3dto2d, AddDim, ToFloat, MaskCompose, MaskToTensor
import bat_functions
from CountLine import CountLine

In [ ]:
import matplotlib.pyplot as plt
im_file = ".../kasanka-bats/frames/17Nov/card-f/GP039791/GP039791_15948.jpg"
im = plt.imread(im_file)

im.shape

In [ ]:
root_output_folder = '.../kasanka-bats/processed/deep-learning/corrected_model'
date = '16Nov'
os.makedirs(root_output_folder, exist_ok=True)

raw_camera_folders = sorted(glob.glob('.../kasanka-bats/gopros/{}/*'.format(date)))

camera_folders = []
for camera_folder in raw_camera_folders:
    videos = sorted(glob.glob(os.path.join(camera_folder, '*.[Mm][Pp]4')))
    camera_name = camera_folder.split('/')[-1]
    if not os.path.exists(os.path.join(root_output_folder, date, camera_name, 'centers.npy')):
        print(*videos, sep='\n')
        print('--------------')
        camera_folders.append(camera_folder)
        



In [ ]:
class BatIterableDataset(IterableDataset):
    def __init__(self, video_files, augmentor=None, max_bad_reads=300):
        self.vid_cap = cv2.VideoCapture(video_files[0])
        self.video_files = video_files
        assert self.vid_cap.isOpened()
        self.more_frames = True
        # How many times a frame can come up false 
        # before assuming end of video
        self.max_bad_reads = max_bad_reads
        self.total_frames_read = 0
        self.total_bad_reads = 0
        self.augmentor = augmentor
        self.video_number = 0
        
    def more_videos(self):
        return self.video_number < len(self.video_files)
    
    def start_next_video(self):
        if self.vid_cap.isOpened():
            self.vid_cap.release()
        self.video_number += 1
        if self.video_number < len(self.video_files):
            print('starting new video')
            print(self.get_read_frame_info())
            self.vid_cap = cv2.VideoCapture(self.video_files[self.video_number])
        
    def video_generator(self):
        while(self.vid_cap.isOpened() or self.more_videos()):
            if not self.vid_cap.isOpened():
                self.start_next_video()
            good_read = False
            num_bad_reads = 0
            while (not good_read and (num_bad_reads < self.max_bad_reads)):
                grabbed, frame = self.vid_cap.read()
                if grabbed:
                    good_read = True
                    self.total_frames_read += 1
                    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    frame = {'image': frame[2:-2, 2:-2]}
                    if np.mean(frame['image'][::50,::50, 2]) < 5:
                        print('too dark')
                        self.vid_cap.release()
                        break
                        
                    if self.augmentor:
                        frame = self.augmentor(frame)
                    yield frame
                else:
                    num_bad_reads += 1
                    self.total_bad_reads += 1
            if not good_read:
                self.vid_cap.release()
                print("video capture closed")
            
    def __iter__(self):
        return self.video_generator()
    
    def __del__(self):
        if self.vid_cap.isOpened():
            self.vid_cap.release()
    
    def is_more_frames(self):
        return self.vid_cap.isOpened()
    
    def get_read_frame_info(self):
        print('{} frames have been read with {} bad reads'.format(self.total_frames_read,
                                                                  self.total_bad_reads))

In [ ]:
folder = './models'

# model_filename = 'model_ThreeLayerWide_epochs_10_batcheff_4_lr_0.05_momentum_0.5_aug_aug-no-blur-2d-17Nov-big-dataset.tar'
model_filename = 'model_UNET_epochs_100_batcheff_16_lr_0.01_momentum_0.9_aug_aug-2d-20Nov-big-dataset.tar'
model_filename = 'model_UNET_epochs_100_batcheff_16_lr_0.01_momentum_0.9_aug_better-norm-aug-2d-20Nov-big-dataset.tar'
model_file = os.path.join(folder, model_filename)
model_file = './models/model_UNETTraditional_epochs_100_batcheff_16_lr_0.01_momentum_0.9_aug_better-norm-aug-2d-20Nov-big-dataset.tar'

In [ ]:
root_train_folder = ".../kasanka-bats/annotations"
mean = np.load(os.path.join(root_train_folder, 'mean.npy'))
std = np.load(os.path.join(root_train_folder, 'std.npy'))

channel = 2
       
    

# augmentor = None
bat_datasets = []
for camera_folder in camera_folders:
    videos = sorted(glob.glob(os.path.join(camera_folder, '*.[Mm][Pp]4')))
    augmentor = MaskCompose([Mask3dto2d(channel_to_use=channel),
                         MaskToTensor(),
                         MaskNormalize(mean[channel]/255, std[channel]/255),
                        ])
    bat_dataset = BatIterableDataset(videos, augmentor=augmentor)
    save_folder = os.path.join(root_output_folder, *camera_folder.split('/')[-2:])
    os.makedirs(save_folder, exist_ok=True)
    os.makedirs(os.path.join(save_folder, 'example-frames'), exist_ok=True)
    bat_datasets.append({'dataset':bat_dataset,
                         'save_folder': save_folder})
                                    

In [ ]:
for i in bat_datasets[0]['dataset']:
    break
# dataloader = data.DataLoader(bat_dataset['dataset'], 
#                                  batch_size=batch_size,
#                                  shuffle=False, num_workers=0, 
#                                  pin_memory=True)

In [ ]:
plt.figure(figsize=(20,20))
# plt.imshow(((np.squeeze(i['image']) - mean) / std)[:,:,1])
plt.imshow(np.squeeze(i['image']))
plt.colorbar()
# plt.figure(figsize=(20,20))


In [ ]:
save_folder

In [ ]:
def logit2prob(logit):
    e_l = np.e ** logit
    return e_l 

def denorm_image(im, mean, std):
    """ Take image the was normalized and return to 0 to 255"""
#     im = np.copy(im)
    im *= std
    im += mean
    im *= 255
    im = np.maximum(im, 0)
    im = np.minimum(im, 255)
    im = im.astype(np.uint8)
    
    return im

should_plot = False
should_save = True

num_classes = 2
bat_prob_thresh = .6
batch_size = 2
early_stop = None
# save some original frames to check detection quality
save_every_n_frames = 1350
channel = 2


device = torch.device("cuda")
model = UNETTraditional(1, 2, should_pad=False)
model.load_state_dict(torch.load(model_file))
model.to(device)

model.train(False)

for bat_dataset in bat_datasets[:]:

    num_frames = 0
    running_loss = 0
    
    print(bat_dataset['save_folder'])

    dataloader = data.DataLoader(bat_dataset['dataset'], 
                                 batch_size=batch_size,
                                 shuffle=False, num_workers=0, 
                                 pin_memory=True)

    centers_list = []
    contours_list = []
    sizes_list = []
    rects_list = []


    for batch_ind, batch in enumerate(dataloader):
        if batch_ind == 0:
            print('started...')
            t0 = time.time()
        if early_stop:
            if batch_ind >= early_stop:
                break

        im_batch = batch['image'].cuda()
    #     masks = batch['mask'].cuda()

        with torch.no_grad():
            outputs = model(im_batch)
            masks = (outputs[:, 1].cpu().numpy() > np.log(bat_prob_thresh)).astype(np.uint8)
            
            for ind, mask in enumerate(masks):
                centers, areas, contours, _, _, rects = bat_functions.get_blob_info(mask)
                centers_list.append(centers)
                sizes_list.append(areas)
                contours_list.append(contours)
                rects_list.append(rects)
                if save_every_n_frames:
                    if num_frames % save_every_n_frames == 0:
                        day = bat_dataset['save_folder'].split('/')[-2]
                        card = bat_dataset['save_folder'].split('/')[-1]
                        im_name = '{}_{}_obs-ind_{}.jpg'.format(day, card, num_frames)
                        im_file = os.path.join(bat_dataset['save_folder'], 
                                               'example-frames', im_name)
                        im = np.squeeze(batch['image'][ind].numpy())
                        im = denorm_image(im, mean[channel]/255, std[channel]/255)
                        cv2.imwrite(im_file, im)
                num_frames += 1


        if should_plot:
            for ind in range(len(im_batch)):

                if 'orig' in batch.keys():
                    plt.figure(figsize=(10,10))
                    plt.imshow(batch['orig'][ind])
                plt.figure(figsize=(10,10))
                im = im_batch[ind].cpu().numpy()
                im = np.transpose(im, (1, 2, 0))
                plt.imshow(im)
                plt.figure(figsize=(10,10))
                im = outputs[ind][0].cpu().numpy()
                plt.imshow(im)
                plt.title('output')
                prob = logit2prob(outputs[ind,1].cpu().numpy())
                mask = (prob > 0.5).astype(np.uint8)

    #             display_im = np.zeros_like(im)
    #             display_im[..., 0] = masks[ind]
                plt.figure(figsize=(10,10))
                plt.imshow(mask)
    #             plt.colorbar()
    #             plt.figure(figsize=(10,10))
    #             plt.imshow(display_im)

    total_time = time.time() - t0
    print(total_time, total_time / batch_ind / batch_size)
    print(bat_dataset['dataset'].get_read_frame_info())
    if should_save:
        save_folder = bat_dataset['save_folder']
        num_contour_files = 15
        file_num = 0
        new_contours = []
        for frame_ind, cs in enumerate(contours_list):
            if frame_ind % int(len(contours_list)/num_contour_files) == 0:
                # start new file
                file_name = f'contours-compressed-{file_num:02d}.npy'
                file = os.path.join(save_folder, file_name)
                np.save(file, np.array(new_contours, dtype=object))
                new_contours = []
                file_num += 1
            new_contours.append([])
            for c in cs:
                cc	= np.squeeze(cv2.approxPolyDP(c, 0.1, closed=True))
                new_contours[-1].append(cc)
        file_name = f'contours-compressed-{file_num:02d}.npy'
        file = os.path.join(save_folder, file_name)
        np.save(file, np.array(new_contours, dtype=object))
#         np.save(os.path.join(save_folder, 'contours.npy'), contours_list)
        np.save(os.path.join(save_folder, 'size.npy'), sizes_list)
        np.save(os.path.join(save_folder,'rects.npy'), rects_list)
        np.save(os.path.join(save_folder, 'centers.npy'), centers_list)

In [ ]:
print(len(contours_list))

In [ ]:
plt.figure(figsize=(20,20))
im = plt.imread('.../kasanka-bats/processed/deep-learning/19Nov/BBC/example-frames/19Nov_BBC_obs-ind_0.jpg')
plt.imshow(im, vmin=0, vmax=255)

In [ ]:
import sys
sys.getsizeof(contours[50000])/1000000

In [ ]:
track_folder = ".../kasanka-bats/processed/deep-learning"
day = "18Nov"

In [ ]:
camera_folders = glob.glob(os.path.join(track_folder, day, '*'))

In [ ]:
def get_obs_ind_from_file(elem):
    return int(elem.split('_')[-1].split('.')[0])

for camera_folder in camera_folders[:1]:
    contours = np.load(os.path.join(camera_folder, 'contours.npy'), allow_pickle=True)
#     centers = np.load(os.path.join(camera_folder, 'centers.npy'), allow_pickle=True)
#     rects = np.load(os.path.join(camera_folder, 'rects.npy'), allow_pickle=True)
#     sizes = np.load(os.path.join(camera_folder, 'size.npy'), allow_pickle=True)
#     frame_folder = os.path.join(camera_folder, 'example-frames')
#     frame_files = sorted(glob.glob(os.path.join(frame_folder, '*.jpg')), key=get_obs_ind_from_file)

In [ ]:
num = [len(s) for s in sizes]

In [ ]:
plt.plot(num[:])

In [ ]:
example_frame_ind = 30
obs_ind = frame_files[example_frame_ind].split('_')[-1].split('.')[0]
plt.figure(figsize=(20,20))
im = plt.imread(frame_files[example_frame_ind])
plt.imshow(im)

In [ ]:
centers[1000].shape

In [ ]:
contours_test = np.copy(contours)

In [ ]:
for ind, (c, c_test) in enumerate(zip(contours, contours_test)):
    assert(len(c) == len(c_test)), print(ind) 
    

In [ ]:
len(contours_test[15001]), len(contours[15001])

In [ ]:
frames = sorted(glob.glob('.../bats/17Nov/Chyniangale/*/*.jpg'))
print(len(frames))

In [ ]:
len(centers[1])

In [ ]:
start_ind = 0
for ind, (con, cent, rect, size) in enumerate(zip(contours[start_ind:], centers[start_ind:], rects[start_ind:], sizes[start_ind:])):
    lengths = [len(con), len(cent), len(rect), len(size)]
    if not np.all(np.array(lengths) == len(con)):
        print('error', ind, lengths)
    if ind == 50000:
        break

In [ ]:
type(contours[0][0])

In [ ]:
frame_ind = 15000
frame = plt.imread(frames[frame_ind])
plt.figure(figsize=(20,20))
plt.imshow(frame)
for position in centers[frame_ind]:
    plt.scatter(position[0], position[1], s=1)

In [ ]:
def test(list_test, array_test):
    l = list_test
    l = l[:4]
    a = array_test
    a = a[:4]
    
    print(a)
    
list_example = [0, 1, 2, 3, 4, 5, 6]
array_example = np.array([0, 1, 2, 3, 4 ,5, 6])

test(list_example, array_example)
print(list_example)
print(array_example)

In [ ]:
max_frame = 60000
first_frame = 100
t0 = time.time()
tracks_list= bat_functions.find_tracks(first_frame, centers, contours, sizes, max_frame=max_frame)
total_time = time.time() - t0
print('total time: {}, fps: {}'.format(total_time, (max_frame - first_frame)/total_time))

In [ ]:
def find_tracks(detection_dict):
    centers = detection_dict['centers']
    contours = detection_dict['contours']
    sizes = detection_dict['sizes']
    max_frame = detection_dict['max_frame']
    save_file = detection_dict['save_file']
    t0 = time.time()
    tracks_list = bat_functions.find_tracks(0, centers, contours, sizes, max_frame=max_frame)
    np.
    total_time = time.time() - t0
    print('total time: {}, fps: {}'.format(total_time, (max_frame - first_frame)/total_time))

In [ ]:
image_file = '.../kasanka-bats/processed/17Nov/Chyniangale/GH040006/GH040006_new03-9500-rects.jpg'
images = [plt.imread(image_file)]

middle_line = CountLine(int(images[0].shape[0]/2), total_frames=max_frame)

forward_size = []
forward_mean_size = []
time_forward = []
backward_size = []
backward_mean_size = []
time_backward = []


for track_ind, track in enumerate(tracks_list[:]):
    result, frame_num = middle_line.is_crossing(track, track_ind)
    if result == 1:
        tracks_list[track_ind]['crossed'] = frame_num
        forward_size.append(tracks_list[track_ind]['size'])
        forward_mean_size.append(np.nanmean(tracks_list[track_ind]['size']))
        time_forward.append(frame_num)
    elif result == -1:
        tracks_list[track_ind]['crossed'] = -frame_num
        backward_size.append(tracks_list[track_ind]['size'])
        backward_mean_size.append(np.nanmean(tracks_list[track_ind]['size']))
        time_backward.append(frame_num)

In [ ]:
np.save(os.path.join(camera_folder, 'test_tracks.npy'), tracks_list)

In [ ]:
test = np.load('test')

In [ ]:
plt.scatter(np.arange(max_frame-first_frame), np.cumsum(middle_line.num_crossing[first_frame:max_frame]))

In [ ]:
track_ind = 241
plt.plot(forward_size[track_ind])
plt.plot(np.ones(len(forward_size[track_ind]))*forward_mean_size[track_ind])

In [ ]:
# plt.hist(forward_size)
plt.plot(forward_size[0])
plt.plot(np.ones(len(forward_size[0]))*forward_mean_size[0])
plt.figure()
plt.hist(forward_mean_size, label='leaving bats')

plt.hist(backward_mean_size, label='coming bats')
plt.legend()

plt.figure()
plt.scatter(time_forward, forward_mean_size)
plt.scatter(time_backward, backward_mean_size)

In [ ]:
t0 = time.time()
frames = []
max_frames = 250

for frame_ind, frame in enumerate(train_dataloader):
    if frame_ind == 0:
        print('starting')
    frames.append(frame)
    if frame_ind > max_frames:
        break
total_time = time.time() - t0
print(total_time, total_time/max_frames, len(frames))
print(bat_dataset.total_frames_read, bat_dataset.total_bad_reads)

In [ ]:
frames[0].shape

In [ ]:
frame_ind

In [ ]:
plt.imshow(frames[-1])
plt.figure()
plt.imshow(frames[0] - frames[-1])

In [ ]:
cap = cv2.VideoCapture(videos[0], cv2.CAP_FFMPEG)
t0 = time.time()

frame_count = 0
max_frames = 100
frames = []

while(True):
    if frame_count > max_frames:
        break
    # Capture frame-by-frame
#     ret, frame = cap.read()
    frames.append(frame)
    ret = cap.grab()
#     ret, frame = cap.retrieve()
    frame_count += 1
#     plt.figure()
#     print(type(frame))
#     plt.imshow(np.array(frame))

#     # Our operations on the frame come here
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#     # Display the resulting frame
#     cv2.imshow('frame',gray)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break
total_time = time.time() - t0
print(total_time, total_time / len(frames), 'frames: {}'.format(len(frames)))

print()
# When everything done, release the capture
cap.release()

In [ ]:
plt.imshow(frame)

In [ ]:
with av.open(filename, metadata_errors="ignore") as container:
    if container.streams.video:
        video_frames = _read_from_stream(
            container,
            start_pts,
            end_pts,
            pts_unit,
            container.streams.video[0],
            {"video": 0},
        )
        video_fps = container.streams.video[0].average_rate
        # guard against potentially corrupted files
        if video_fps is not None:
            info["video_fps"] = float(video_fps)

In [ ]:
class BatInferenceDataset(VisionDataset):
    """ Dataset for running inference on bat videos.
    
    Args:
        root (string): Root directory of the videos.
        batch_size (int): number of frames to include in batch.
        transform (callable, optional): A function/transform that takes in a TxHxWxC video
            and returns a transformed version.

    Returns:
        video (Tensor[T, H, W, C]): the `T` video frames
        video_idx (int): the index of the video the clip is from
        clip_idx (int): the index of the video clip"""
    
    def __init__(self, root, batch_size, frame_rate=None, transform=None, num_workers=1):

        video_paths = glob.glob('./*.mp4')
        print('num workers', num_workers)
        print(video_paths)

        video_clips = VideoClips(
            video_paths,
            batch_size,
            frames_between_clips=1,
            frame_rate=frame_rate,
            num_workers=num_workers
        )
        self.video_clips_metadata = video_clips.metadata
        self.transform = transform
        self.video_clips = video_clips
        
    @property
    def metadata(self):
        return self.video_clips_metadata

    def __len__(self):
        return self.video_clips.num_clips()

    def __getitem__(self, idx):
        video, audio, _, video_idx = self.video_clips.get_clip(idx)
        
        item = {'image': video, 'video_idx': video_idx, 'idx': idx}

        if self.transform is not None:
            item = self.transform(item)

        return item

In [ ]:
type(test_dataset.video_clips.get_clip(0)[0])

In [ ]:
# mean = np.load(os.path.join(root_train_folder, 'mean.npy'))
# std = np.load(os.path.join(root_train_folder, 'std.npy'))

folder = './models'

model_filename = 'model_ThreeLayerWide_epochs_10_batcheff_4_lr_0.05_momentum_0.5_aug_aug-no-blur-2d-17Nov-big-dataset.tar'
model_file = os.path.join(folder, model_filename)

In [ ]:
folder = '.../kasanka-bats/gopros/17Nov/card-b'
folder = "."
videos = glob.glob(os.path.join(folder, '*.mp4'))
print(videos)

In [ ]:
import time
t0 = time.time()
clips = VideoClips(videos, 4, frames_between_clips=1, 
                   frame_rate=None,num_workers=7)
print(time.time() - t0)

In [ ]:
504

In [ ]:
def with_opencv(filename):
    import cv2
    video = cv2.VideoCapture(filename)

    duration = video.get(cv2.CAP_PROP_POS_MSEC)
    frame_count = video.get(cv2.CAP_PROP_FRAME_COUNT)

    return duration, frame_count

In [ ]:
import time

video_path = "."
batch_size = 4
frame_rate = 30
num_workers = 0

transforms = MaskCompose([Transform3dto2d(channel_to_use=2),
                          AddDim(new_dim=1),
                          ToFloat()
                         ])

t0 = time.time()


test_dataset = BatInferenceDataset(video_path, batch_size, frame_rate, 
                                   transform=transforms,
                                   num_workers=num_workers
                                  )
print(time.time() - t0)

In [ ]:
train_dataloader = data.DataLoader(test_dataset, batch_size=None,
                                   shuffle=False, num_workers=7, pin_memory=False)

In [ ]:
for idx, item in enumerate(train_dataloader):
    print(idx)
    if idx > 30:
        break

In [ ]:
item['image'].shape
# item['image'].type()

In [ ]:
def logit2prob(logit):
    e_l = np.e ** logit
    return e_l 

should_plot = True

num_classes = 2

conf_matrix = np.zeros((num_classes+1, num_classes+1), dtype=np.int64)

device = torch.device("cuda")
model = ThreeLayerSemSegNetWideView(1, 2)
model.load_state_dict(torch.load(model_file))
model.to(device)

model.train(False)

num_frames = 0

running_loss = 0

dataloader = train_dataloader


num_batches = 1
for batch_ind, batch in enumerate(dataloader):
    
    
    if batch_ind >= num_batches and should_plot:   
        break
        
    im_batch = batch['image'].cuda()
#     masks = batch['mask'].cuda()
    
    with torch.no_grad():
        outputs = model(im_batch)
        num_frames += len(im_batch)
#     loss = loss_fn(outputs, masks)
#     running_loss += loss.item()*dataloader.batch_size
    
#     outputs = outputs.cpu().numpy()
#     preds = np.argmax(outputs, axis=1)
#     masks = batch['mask'].numpy()
    
    
    
#     conf_matrix = get_conf_matrix(conf_matrix, num_classes, 
#                                   preds, masks)
    
    if should_plot:
        for ind in range(len(im_batch)):

            if 'orig' in batch.keys():
                plt.figure(figsize=(10,10))
                plt.imshow(batch['orig'][ind])
            plt.figure(figsize=(10,10))
            im = im_batch[ind].cpu().numpy()
            im = np.transpose(im, (1, 2, 0))
            plt.imshow(im)
            plt.figure(figsize=(10,10))
            im = outputs[ind][0].cpu().numpy()
            plt.imshow(im)
            plt.title('output')
            
#             display_im = np.zeros((masks[ind].shape[0], masks[ind].shape[1], 3))
#             display_im[..., 0] = masks[ind]
# #             display_im[..., 1] = logit2prob(outputs[ind,1])
#             display_im[..., 1] = np.argmax(outputs[ind], axis=0)

            
#             plt.colorbar()
#             plt.figure(figsize=(10,10))
#             plt.imshow(display_im)

In [ ]:
max_frame = 100
t0 = time.time()
for frame_idx, (batch, video_idx, idx) in enumerate(train_dataloader):
    if frame_idx != idx:
        print(frame_idx, idx)
    if frame_idx > max_frame:
        break
total_time = time.time() - t0
print(total_time, total_time / max_frame)
    

In [ ]:
date = '17Nov'
camera_ind = 0

kasanka_folder =  '.../kasanka-bats/gopros'

camera_folder = sorted(glob.glob(os.path.join(kasanka_folder, date, '*')))[camera_ind]

camera_files = sorted(glob.glob(os.path.join(camera_folder, '*.MP4')))

In [ ]:
camera_files

In [ ]:
frames, _, info = torchvision.io.video.read_video(camera_files[0])

In [ ]:

import torch, torchvision
torch.__version__, torchvision.__version__

In [ ]:
from torchvision.datasets.utils import download_url
download_url("https://github.com/pytorch/vision/blob/master/test/assets/videos/WUzgd7C1pWA.mp4?raw=true", ".", "WUzgd7C1pWA.mp4")

In [ ]:
video_path = "./WUzgd7C1pWA.mp4"

from torchvision.datasets.video_utils import VideoClips
video_clips = VideoClips([video_path], clip_length_in_frames=32, frames_between_clips=32)

In [ ]:
video_clips.num_clips()

In [ ]:
vid, _, info, video_idx = video_clips.get_clip(0)

In [ ]:
vid.shape